In [59]:
import numpy as np # linear algebra
import pandas as pd
import tensorflow as tf


**IMPORT ALL SEASONS CSV IN DATAFRAM**

In [60]:
s1_transcript = pd.read_csv('../input/house-md-transcripts/season1.csv', encoding= 'unicode_escape')
s2_transcript = pd.read_csv('../input/house-md-transcripts/season1.csv', encoding= 'unicode_escape')
s3_transcript = pd.read_csv('../input/house-md-transcripts/season1.csv', encoding= 'unicode_escape')
s4_transcript = pd.read_csv('../input/house-md-transcripts/season1.csv', encoding= 'unicode_escape')
s5_transcript = pd.read_csv('../input/house-md-transcripts/season1.csv', encoding= 'unicode_escape')
s6_transcript = pd.read_csv('../input/house-md-transcripts/season1.csv', encoding= 'unicode_escape')
s7_transcript = pd.read_csv('../input/house-md-transcripts/season1.csv', encoding= 'unicode_escape')

In [61]:
all_transcripts = [s1_transcript,
                    s2_transcript, s3_transcript, s4_transcript,
                  s5_transcript, s6_transcript, s7_transcript]

In [62]:
all_transcripts

In [63]:
for season, transcript in enumerate(all_transcripts, start=1):
     # add a column for the season to each transcript
     transcript['season'] = season
     # clear whitespace from names so we don't have 'House' and 'House '
     transcript['name'] = transcript['name'].str.strip()

In [64]:
transcript

In [65]:
house=pd.concat(all_transcripts)


In [66]:
house.head()

In [67]:
house['text'] = house["name"].astype(str) + " : " + house["line"].astype(str)


In [68]:
house.to_csv('./house_md_transcript_new.csv')


In [69]:
house=house.drop("name",axis=1)


In [70]:
house=house.drop("line",axis=1)
house=house.drop("season",axis=1)

In [71]:
house["text"]=house["text"].astype(str)


In [72]:
house["text"].to_csv('./house_md_transcript_new.txt')


In [73]:
house_2='./house_md_transcript_new.txt'


In [74]:
# Read, then decode for py2 compat.

text = open(house_2, 'rb').read().decode(encoding='utf-8')

# length of text is the number of characters in it

print ('Length of text: {} characters'.format(len(text)))

In [75]:
# Take a look at the first 250 characters in text
print(text[:250])

In [76]:
# The unique characters in the file

vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

In [77]:
# process the text

char2index = {u:i for i, u in enumerate(vocab)}
index2char = np.array(vocab)
text_as_int = np.array([char2index[c] for c in text])

In [78]:
print('{')
for char,_ in zip(char2index, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2index[char]))
print('  ...\n}')

In [79]:
# Show how the first 13 characters from the text are mapped to integers

print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

In [80]:
# convert text vector to charcter indices

# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

In [81]:
# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
      print(index2char[i.numpy()])

In [82]:
# convert character to sequence

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(index2char[item.numpy()])))

In [83]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [84]:
# print input and target values

for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(index2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(index2char[target_example.numpy()])))

In [85]:
# Batch size
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

In [86]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 256

In [87]:
import os
# Directory where the checkpoints will be saved
checkpoint_dir = 'training_checkpoints'
import shutil
try:
    shutil.rmtree(checkpoint_dir)
except:
    print("directory not used yet.")

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    monitor='loss',
    save_weights_only=True, 
    save_best_only=True)

In [88]:

# build the model

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[BATCH_SIZE, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
])
model.summary()

In [89]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

In [90]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

In [91]:
#Perplexity function to evaluate model performance
def perplexity(y_true, y_pred):
    cross_entropy = tf.keras.backend.mean(tf.keras.backend.sparse_categorical_crossentropy(y_true, y_pred))
    perplexity = tf.keras.backend.exp(cross_entropy)
    return perplexity

In [92]:
model.compile(optimizer='adam', loss=loss,metrics=[perplexity])


In [93]:
# Directory where the checkpoints will be saved
import os
checkpoint_dir = './training_checkpoints'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [94]:
history = model.fit(dataset, epochs=100, callbacks=[checkpoint_callback])

In [95]:
# to keep prediction step simple we use batch size of 1. for that we need to rebuild model
# and restore weight from checkpoint

tf.train.latest_checkpoint(checkpoint_dir)

**CREATING THE LSTM MODE**

In [96]:
# rebuild the model

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[1, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
])
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

In [97]:
def generate_text(model, start_string):
    # Number of characters to generate
    num_generate = 5000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2index[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(index2char[predicted_id])

    return (start_string + ''.join(text_generated))


**OUTPUT WITH DIALOUGES**

In [98]:
print(generate_text(model, start_string=u"Melanie: I don't doubt it, no bus stops near Brad's. You spent the night, the alarm didn't work. Or maybe it did. "))
